# Sample text corpus processing

In [3]:
%%html
<style>
.jp-RenderedHTMLCommon td, .jp-RenderedHTMLCommon th, .jp-RenderedHTMLCommon tr {
    text-align: left;
    vertical-align: top;
}
</style>

## Statens offentliga utredningar (SOU)

- KB har digitaliserat alla SOU:er publicerade mellan **1922 och 1999** (1129 stycken).
- Utredningarna kan laddas ned från **http://regina.kb.se/sou/** i PDF-format.
- Filerna kan vara rätt stora, flera hundra MB, då de innehåller dels alla sidor scannade.
- De innehåller även inbäddad **OCR:ad text** vilket möjliggör textsökning.
- OCR:a text är genomgående av mycket **hög kvalitet**.

[<img src="./images/kb_regina_sou.jpg" alt="http://regina.kb.se/sou/" width="700px"/>](http://regina.kb.se/sou/)


### The scanned documents

| SOU 1933:15        | Page 23 | 
| ------------- |:-------------|
| [<img src="./images/kb-digark-2106487-page-0.jpg" alt="1933:15 Utredning och förslag angående importmonopol på kaffe" width="400px"/>](https://weburn.kb.se/sou/211/urn-nbn-se-kb-digark-2106487.pdf)      | <img src="./images/kb-digark-2106487-page-23.jpg" alt="1933:15 Page 23" width="400px"/> |

### The OCR:ed text is of very high quality

> 23 Enligt dessa siffror skulle sålunda **uuder** åren 1911—1913 43 % och år
1931 65 % av Sveriges import av orostat kaffe ha varit av brasilianskt
ursprung. Ingendera av dessa relationer är emellertid riktig, den första
dock i vida mindre mån än den sista. Före kriget var Tyskland ännu en
stor mellanhand för vår kaffehandel, liksom i viss utsträckning även
Danmark och Holland, Före 1905 voro dessa länder, framför allt Tysk-
land, såvitt kan slutas av den svenska handelsstatistiken, ännu mera do-
minerande — vi köpte vid denna tid så gott som allt vårt kaffe genom
affärshus i Hamburg, Rotterdam, Köpenhamn o. s. v. Efter tillkomsten
år 1904 av ett nytt svenskt rederiföretag med trafik på Syd- och Central-
amerika har bladet vänt sig. Redan år 1905 återfinnas Brasilien och
Centralamerika i vår importstatistik med införselsiffror av resp. 4-5 och
0\*3 milj. kg. från att förut ha varit obefintliga, och sedan dess har den
direkta införseln oavlåtligt stigit på den indirektas bekostnad. År 1931
redovisas enligt tab. 15 den direkta importen till 85 % av den totala och
importen från Brasilien till 65 %. I verkligheten torde omkr. 70—75 %
av vårt kaffe vara av brasilianskt ursprung. Danmark, Holland, Tysk-
land och England förmedla väsentligen vår import av dyrare kaffesor-
ter från Guatemala (vars export till stor del behärskas av den tyska
handeln), Mexiko, Costa Rica, Haiti, Java o. s. v.

### But tables, indexes tec. are problematic

> T a b . 16. E x p o r t å r 1930 av brasilianskt kaffe.
U. S. A
Frankrike
Tyskland
Holland
Italien
Argentina • • • -
Sverige
Belgien
Övriga
Summa
1 000 säckar
å 60 kg.
8 006
1995
912
862
781
482
444
410
1399
15 291
52-4
13-o
6-0
5-6
5 i
3-2
2-9
27
9 1
lOOo

### TRY IT: Clean up the text in SOU 1933:15
- Typical text preprocessing includes removal of frequent words, short words, etc.
- Also common to filter based part-of-speech tags

In [2]:
# folded code
# -*- coding: utf-8 -*-
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import re
import string

punct_table = str.maketrans('', '', string.punctuation)

%matplotlib inline

hyphen_regexp = re.compile(r'\b(\w+)-\s*\r?\n\s*(\w+)\b', re.UNICODE)

def read_text_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        document = f.read()
    return document

def tokenize_and_sanitize(document, de_hyphen=True, min_length=3, only_isalpha=True, remove_puncts=True, to_lower=True, remove_stop=True):
        
    # hantera avstavningar
    if de_hyphen:
        document = re.sub(hyphen_regexp, r"\1\2\n", document)

    tokens = nltk.word_tokenize(document)
    
    # Ta bort ord kortare än tre tecken
    if min_length > 0:
        tokens = [ x for x in tokens if len([w for w in x if w.isalpha()]) > min_length ]
        
    # Ta bort ord som inte innehåller någon siffra eller bokstav
    if only_isalpha:
        tokens = [ x for x in tokens if x.isalpha() ]

    if remove_puncts:
        tokens = [ x.translate(punct_table) for x in tokens ]

    # Transformera till små bokstäver
    if to_lower:
        tokens = [ x.lower() for x in tokens ]
        
    # Ta bort de vanligaste stoporden
    if remove_stop:
        stopwords = nltk.corpus.stopwords.words('swedish')
        tokens = [ x for x in tokens if x not in stopwords ]
        
    return [ x for x in tokens if len(x) > 0 ]

def plot_xy_data(data, title='', xlabel='', ylabel='', **kwargs):
    
    x = list(data[0])
    y = list(data[1])
    labels = x

    plt.figure(figsize=(8, 8 / 1.618))
    plt.plot(x, y, 'ro', **kwargs)
    plt.xticks(x, labels, rotation='45')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    plt.show()
    
container=dict(
    display_type=widgets.Dropdown(
        description='Show',
        value='statistics',
        options={
            'Source text': 'source_text',
            'Sanitized text': 'sanitized_text',
            'Statistics': 'statistics'           
    }),
    min_length=widgets.IntSlider(value=0, min=0, max=5, step=1, description='Min alpha', tooltip='Min number of alphabetic characters'),
    de_hyphen=widgets.ToggleButton(value=False, description='Dehyphen', disabled=False, tooltip='Fix hyphens', icon=''),
    to_lower=widgets.ToggleButton(value=False, description='Lowercase', disabled=False, tooltip='Transform text to lowercase', icon=''),
    remove_stop=widgets.ToggleButton(value=False, description='No stopwords', disabled=False, tooltip='Remove stopwords', icon=''),
    only_isalpha=widgets.ToggleButton(value=False, description='Only alpha', disabled=False, tooltip='Keep only alphabetic words', icon=''),
    remove_puncts=widgets.ToggleButton(value=False, description='Remove puncts.', disabled=False, tooltip='Remove punctioations characters', icon=''),
    progress=widgets.IntProgress(value=0, min=0, max=5, step=1, description='' )
)

output1 = widgets.Output() #layout={'border': '1px solid black'})
output2 = widgets.Output() #layout={'border': '1px solid black'})
default_output = None

tokens = []

def display_document(display_type, to_lower, remove_stop, only_isalpha, remove_puncts, min_length, de_hyphen, word_count=500):

    global tokens
    
    p =  container['progress']
    p.value = 0
    try:
        output1.clear_output()
        output2.clear_output()
        default_output.clear_output()
        document = read_text_file('./data/urn-nbn-se-kb-digark-2106487.txt')
        p.value = p.value + 1

        if display_type == 'source_text':
            # Utskrift av de första och sista 250 tecknen:
            with output1:
                print('{}\n.................\n(NOT SHOWN TEXT)\n.................\n{}'.format(document[:2500], document[-250:]))
            p.value = p.value + 1
            return
        
        p.value = p.value + 1

        tokens = tokenize_and_sanitize(
            document,
            de_hyphen=de_hyphen,
            min_length=min_length,
            only_isalpha=only_isalpha,
            remove_puncts=remove_puncts,
            to_lower=to_lower,
            remove_stop=remove_stop
        )

        if display_type in ['sanitized_text', 'statistics']:
            
            p.value = p.value + 1
            
            if display_type == 'sanitized_text':
                with output1:
                    display('{}\n.................\n(NOT SHOWN TEXT)\n.................\n{}'.format(
                        ' '.join(tokens[:word_count]),
                        ' '.join(tokens[-word_count:])
                    ))
                p.value = p.value + 1
                return
            
            if display_type == 'statistics':

                wf = nltk.FreqDist(tokens)
                p.value = p.value + 1
                
                with output1:
                    
                    df = pd.DataFrame(wf.most_common(25), columns=['token','count'])
                    display(df)
                
                with output2:
                    
                    print('Antal ord (termer): {}'.format(wf.N()))
                    print('Antal unika termer (vokabulär): {}'.format(wf.B()))
                    print(' ')
                    
                    data = list(zip(*wf.most_common(25)))
                    plot_xy_data(data, title='Word distribution', xlabel='Word', ylabel='Word count')
                    
                    wf = nltk.FreqDist([len(x) for x in tokens])
                    data = list(zip(*wf.most_common(25)))
                    plot_xy_data(data, title='Word length distribution', xlabel='Word length', ylabel='Word count')
    
    except Exception as ex:
        raise
        
    finally:
        p.value = 0

i_widgets = widgets.interactive(display_document, **container)
default_output = i_widgets.children[-1]
display(widgets.VBox([
    widgets.HBox([
        container['display_type'],
        container['to_lower'],
        container['remove_stop'],
        container['de_hyphen'],
        container['only_isalpha'],
        container['remove_puncts']
    ]),
    widgets.HBox([container['min_length'], container['progress']]),
    widgets.HBox([output1, output2]),
    default_output
]))

i_widgets.update()


### TRY IT: Part-Of-Speech tagging (POS) using Språkbanken Sparv

> - Part-of-speech tagging (POS) is the process of **assigning word classes** to each word.
> - The **Språkbanken Sparv** API is an online tool
> - Sparv does a lot more: tokenization, dependency analysis, lemmatization, named entity recognition, sentiment analysis, ...
> - Sparv uses **HunPOS** (insert reference) for POS tagging
> - POS tagging is a** common NLP task** and can be used to improve quality of various NLP methods.
> - POS tag set: https://spraakbanken.gu.se/korp/markup/msdtags.html 

In [3]:
# folded code
import requests
import json
import ipywidgets as widgets

settings={
    "corpus": "exempelkorpus",
    "lang": "sv",
    "textmode": "plain",
    "word_segmenter": "default_tokenizer",
    "sentence_segmentation": {
        "sentence_chunk": "paragraph",
        "sentence_segmenter": "default_tokenizer"
    },
    "paragraph_segmentation": {
        "paragraph_segmenter": "blanklines"
    },
    "positional_attributes": {
        "lexical_attributes": [ "pos", "msd", "lemma", ],
        "compound_attributes": [ ],
        "dependency_attributes": [ ],
        "sentiment": [ ]
    },
    "named_entity_recognition": [ ],
    "text_attributes": {
        "readability_metrics": [ ]
    }
}

headers = {
    "Accept": "application/xml",
}

cc=dict(
    text_widget = widgets.Textarea(
        value='Detta är ett bra exempel på en text som kan annoteras via Sparv.',
        placeholder='Type some',
        description='Text:',
        continuous_update=False,
        layout=widgets.Layout(width="400px")
    ),
    button=widgets.Button(
        description='Annotate it!',
        disabled=False,
        button_style='',
        tooltip='Use Språkbanken Sparv to annotate text!',
        icon=''
    ),
    progress=widgets.IntProgress(value=0, min=0, max=3, step=1, description='' )
)

cc_output = widgets.Output() #layout={'border': '1px solid black'})

def call_spraakbanken(text):
    p = cc['progress']
    try:
        cc_output.clear_output()
        data = {
            'text': text,
        } 
        url = "https://ws.spraakbanken.gu.se/ws/sparv/v2/?settings={}".format(json.dumps(settings))

        p.value = 1
        response = requests.post(url=url, headers=headers, data=data) 
        p.value = 2
        with cc_output:
            print(response.text)
        p.value = 3
    except Exception as ex:
        raise
    finally:
        p.value = 0
        
from IPython.display import display

def on_button_clicked(b):
    text = cc['text_widget'].value
    call_spraakbanken(text)

cc['button'].on_click(on_button_clicked)

display(widgets.VBox([
    widgets.HBox([cc['text_widget'], cc['button'], cc['progress'] ]),
    cc_output
]))


### TRY IT: Språkbanken NER tagging av SOU

In [4]:
import nltk
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib inline

import pandas as pd

entities = pd.read_csv('./data/SOU_1990_total_ner_extracted.csv', sep='\t',
                       names=['filename', 'year', 'location', 'categories', 'entity'])

entities['document_id'] = entities.filename.apply(lambda x: int(x.split('_')[1]))
entities['categories'] = entities.categories.str.replace('/', ' ')
entities['category'] = entities.categories.str.split(' ').str.get(0)
entities['sub_category'] = entities.categories.str.split(' ').str.get(1)

entities.drop(['location', 'categories'], inplace=True, axis=1)

document_names = pd.read_csv('./data/SOU_1990_index.csv',
                             sep='\t',
                             names=['year', 'sequence_id', 'report_name']).set_index('sequence_id')

def plot_freqdist(wf, n=25, **kwargs):
    data = list(zip(*wf.most_common(n)))
    x = list(data[0])
    y = list(data[1])
    labels = x

    plt.figure(figsize=(13, 13/1.618))
    plt.plot(x, y, '--ro', **kwargs)
    plt.xticks(x, labels, rotation='45')
    plt.show()

doc_names = { v: k for k, v in document_names.report_name.to_dict().items()}
doc_names['** ALL DOCUMENTS **'] = 0
@widgets.interact(category=entities.category.unique())
def display_most_frequent_pos_tags(document_id=doc_names, category='LOC', top=10):
    global entities
    locations = entities
    if document_id > 0:
        locations = locations.loc[locations.document_id==document_id]
    locations = locations.loc[locations.category==category]['entity']
    location_freqs = nltk.FreqDist(locations)
    #location_freqs.tabulate()
    plot_freqdist(location_freqs, n=top)

# display_most_frequent_pos_tags()

interactive(children=(Dropdown(description='document_id', options={'Stadsregioner i Europa': 34, 'Den elintens…